In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [4]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

In [5]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [6]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [7]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [10]:
## Task 1: Mark all launch sites on a map
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label

import folium

# Create a map centered around Houston
site_map = folium.Map(location=[29.7604, -95.3698], zoom_start=5)

# Define launch sites and their coordinates
launch_sites = {
    'CCAFS LC-40': {'location': [28.5623, -80.5774], 'marker_color': 'blue'},
    'CCAFS SLC-40': {'location': [28.5632, -80.5768], 'marker_color': 'blue'},
    'KSC LC-39A': {'location': [28.5733, -80.6469], 'marker_color': 'blue'},
    'VAFB SLC-4E': {'location': [34.6328, -120.6107], 'marker_color': 'blue'}
}

# Add circle and marker for each launch site
for site, info in launch_sites.items():
    folium.Circle(
        location=info['location'],
        radius=1000,
        color=info['marker_color'],
        fill=True
    ).add_to(site_map)
    
    folium.Marker(
        location=info['location'],
        icon=folium.Icon(color=info['marker_color']),
        popup=site
    ).add_to(site_map)

# Display the map
site_map


In [15]:
# # Task 2: Mark the success/failed launches for each site on the map
# Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates. Recall that data frame spacex_df has detailed launch records, and the class column indicates if this launch was successful or not

# # Iterate through the DataFrame and add markers for each launch outcome
# for index, row in spacex_df.iterrows():
#     site = row['LaunchSite']
#     outcome = row['Class']
#     location = [row['Lat'], row['Long']]
    
#     # Determine marker color based on launch outcome
#     marker_color = 'green' if outcome == 'Success' else 'red'
    
#     # Add marker to the map
#     folium.Marker(
#         location=location,
#         icon=folium.Icon(color=marker_color),
#         popup=outcome
#     ).add_to(site_map)

# # Display the map with launch outcomes marked
# site_map

spacex_df.tail(10)

marker_cluster = MarkerCluster()

# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

In [22]:
## TASK 3: Calculate the distances between a launch site to its proximities
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [23]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance


In [25]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# Replace launch_site_lat, launch_site_lon, coastline_lat, and coastline_lon with actual coordinates
launch_site_lat = 28.5623
launch_site_lon = -80.5774
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate the distance between the launch site and the coastline point
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
print("Distance between launch site and coastline:", distance_coastline, "km")



Distance between launch site and coastline: 0.5839189140354601 km


In [28]:
from folium import DivIcon

# Replace coastline_lat and coastline_lon with the actual coordinates of the closest coastline point
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate the distance between the launch site and the coastline point
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Create a Folium Marker for the closest coastline point
coastline_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_coastline),
    )
)

# Add the marker to the map
site_map.add_child(coastline_marker)

# Display the map
site_map


In [30]:
# Create a PolyLine between the launch site and the closest coastline point
folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
    color='blue'
).add_to(site_map)

# Display the map with the PolyLine
site_map

In [31]:
# TODO: Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use MousePosition to find the their coordinates on the map first

#A railway map symbol may look like this:
# Replace these placeholder values with the actual coordinates obtained from MousePosition
closest_city_lat = 28.5721
closest_city_lon = -80.648

# Calculate the distance between the launch site and the closest city
distance_city = calculate_distance(launch_site_lat, launch_site_lon, closest_city_lat, closest_city_lon)

# Create a Folium Marker for the closest city
city_marker = folium.Marker(
    location=[closest_city_lat, closest_city_lon],
    icon=folium.Icon(color='green'),
    popup='Closest City'
)

# Add the marker to the map
site_map.add_child(city_marker)

# Draw a PolyLine between the launch site and the closest city
folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [closest_city_lat, closest_city_lon]],
    color='green'
).add_to(site_map)

# Display the map
site_map


In [32]:
#A highway map symbol may look like this:
# Replace these placeholder values with the actual coordinates obtained from MousePosition
closest_highway_lat = 28.5721
closest_highway_lon = -80.648

# Create a Folium Marker for the closest highway
highway_marker = folium.Marker(
    location=[closest_highway_lat, closest_highway_lon],
    icon=folium.Icon(color='blue', icon='road', prefix='fa'),
    popup='Closest Highway'
)

# Add the marker to the map
site_map.add_child(highway_marker)

# Draw a PolyLine between the launch site and the closest highway
folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [closest_highway_lat, closest_highway_lon]],
    color='blue'
).add_to(site_map)

# Display the map
site_map


In [34]:
#A city map symbol may look like this:
# Replace these placeholder values with the actual coordinates obtained from MousePosition
closest_city_lat = 28.5721
closest_city_lon = -80.648

# Create a Folium Marker for the closest city
city_marker = folium.Marker(
    location=[closest_city_lat, closest_city_lon],
    icon=folium.Icon(color='orange', icon='building', prefix='fa'),  # Using Font Awesome icon for buildings
    popup='Closest City'
)

# Add the marker to the map
site_map.add_child(city_marker)

# Draw a PolyLine between the launch site and the closest city
folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [closest_city_lat, closest_city_lon]],
    color='orange'  # Use the same color as the city marker
).add_to(site_map)

# Display the map
site_map
